In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    scipy-1.3.2                |   py36h921218d_0        18.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    li

In [2]:
!conda install -c conda-forge BeautifulSoup4 --yes

!conda install -c conda-forge lxml --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    soupsieve-1.9.4            |           py36_0          58 KB  conda-forge
    beautifulsoup4-4.8.1       |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         207 KB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.4-py36_0 conda-forge

The following packages will be UPDATED:

    beautifulsoup4: 4.6.3-py37_0             --> 4.8.1-py36_0 conda-forge


soupsieve-1.9.4     

In [125]:
from bs4 import BeautifulSoup # import the BeautifulSoup library so we can parse HTML and XML documents
import urllib.request # import the library we use to open URLs
import xml.etree.ElementTree as ET
import requests

# specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# parse the HTML from our URL into the BeautifulSoup parse tree format
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table')

#processing the cells that have an assigned borough and Ignoring cells with a borough that is Not assigned.

lst_table = []
for row in table.find_all('tr'):
    if (row.contents[3].text != 'Not assigned'):
        lst_table.append([row.contents[1].text, row.contents[3].text, row.contents[5].text[:-1]])

lst_table

[['Postcode', 'Borough', 'Neighbourhood'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned'],
 ['M9A', "Queen's Park", "Queen's Park"],
 ['M1B', 'Scarborough', 'Rouge'],
 ['M1B', 'Scarborough', 'Malvern'],
 ['M3B', 'North York', 'Don Mills North'],
 ['M4B', 'East York', 'Woodbine Gardens'],
 ['M4B', 'East York', 'Parkview Hill'],
 ['M5B', 'Downtown Toronto', 'Ryerson'],
 ['M5B', 'Downtown Toronto', 'Garden District'],
 ['M6B', 'North York', 'Glencairn'],
 ['M9B', 'Etobicoke', 'Cloverdale'],
 ['M9B', 'Etobicoke', 'Islington'],
 ['M9B', 'Etobicoke', 'Martin Grove'],
 ['M9B', 'Etobicoke', 'Princess Gardens'],
 ['M9B', 'Etobicoke', 'West Deane Park'],
 ['M1C', 'Scarborough', 'Highland Creek'],
 ['M1C', 'Scarborough', 'Rouge Hill'],
 ['M1C', 'Scarborough', 'Port Union'],
 ['M3C', 'North Yor

In [123]:
       
df_postal = pd.DataFrame(lst_table[1:], columns=lst_table[0])
df_code = df_postal['Postcode'].unique()
df_postal.set_index('Postcode', drop=False, inplace=True)

separator=', '
for postcode in df_code:
    df_tmp = df_postal.loc[postcode]
    if(df_tmp.size > 3):
        neighbourhood = separator.join(df_tmp['Neighbourhood'])
        df_postal.loc[postcode, 'Neighbourhood'] = neighbourhood

df_postal.drop(df_postal[df_postal.Neighbourhood == 'Not assigned'].index, inplace=True)
df_postal.drop_duplicates('Postcode', inplace=True)
df_postal.reset_index(drop = True, inplace=True)
df_postal.head(10)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M9A,Queen's Park,Queen's Park
5,M1B,Scarborough,"Rouge, Malvern"
6,M3B,North York,Don Mills North
7,M4B,East York,"Woodbine Gardens, Parkview Hill"
8,M5B,Downtown Toronto,"Ryerson, Garden District"
9,M6B,North York,Glencairn


In [117]:
print('Number of records in the data frame is: ', df_postal.shape)

Number of records in the data frame is:  (102, 3)


In [129]:
#Use geopy library to get the latitude and longitude values of New York City.
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="cot_explorer")
location = geolocator.geocode(address)
print(location)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

Toronto, Golden Horseshoe, Ontario, M6K 1X9, Canada
The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [134]:
D = []
E = []

for index, row in df_postal.iterrows():
    address = row['Borough']
    geolocator = Nominatim(user_agent="cot_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    D.append(format(latitude))
    E.append(format(longitude))
    
df_postal["Latitude"] = D
df_postal["Longtitude"] = E
 

In [133]:
df_postal.head(102)

,Postcode,Borough,Neighbourhood,Latitude,Longtitude
0,M3A,North York,Parkwoods,43.7708175,-79.4132998
1,M4A,North York,Victoria Village,43.7708175,-79.4132998
2,M5A,Downtown Toronto,Harbourfront,43.654027,-79.3802003
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7708175,-79.4132998
4,M9A,Queen's Park,Queen's Park,43.6599803,-79.3903686
5,M1B,Scarborough,"Rouge, Malvern",54.2820009,-0.4011868
6,M3B,North York,Don Mills North,43.7708175,-79.4132998
7,M4B,East York,"Woodbine Gardens, Parkview Hill",43.6913391,-79.3278212
8,M5B,Downtown Toronto,"Ryerson, Garden District",43.654027,-79.3802003
9,M6B,North York,Glencairn,43.7708175,-79.4132998
